# MS Summer Academy NPS Data Exploration (2016-2017)

## Questsions Asked:

    * How many promoters, passives, and detractors are there in both years? How do the scores differ by year?
    * What track of students had the best experience at the summer academy? What about the worst experience?
    * Did students feel as though the pacing increased as the program went on?
    * Which location had the best overall experience?
    * Did students at the NY location have a better or worse expeience as the program went on?
  

# General NPS Cleaning Process:

- Step 1: Import, Clean, and Aggregate Weeks 1-7 for 2016 Data
- Step 2: Import and Clean Week 8 for 2016 Data
- Step 2.5: Aggregate Weeks 1-7 with Week 8 to produce Full 2016 Dataset
- Step 3: Import and Clean 2017 Data
- Step 3.5: Aggregate Full 2016 Data with 2017 Data to produce Complete Dataset

### Promoters, Passives, Detractors for 2017: 78, 34, 8


In [1]:
# Pandas is a library for basic data analysis
import pandas as pd

# NumPy is a library for advanced mathematical computation
import numpy as np

# MatPlotLib is a library for basic data visualization
import matplotlib.pyplot as plt

# SeaBorn is a library for advanced data visualization
import seaborn as sns

import glob

## _Stretch Challenge_:

### Functionalize Data Manipulation code!

In [2]:
sns.set(style="white", context="notebook", palette="deep")

COLOR_COLUMNS = ["#66C2FF", "#5CD6D6", "#00CC99", "#85E085", "#FFD966", "#FFB366", "#FFB3B3", "#DAB3FF", "#C2C2D6"]

sns.set_palette(palette=COLOR_COLUMNS, n_colors=4)

---

# Data Cleaning and Aggregation

In [3]:
REL_PATH_DIRECTORY = "../datasets/SA_Feedback_Surveys_FINAL/2016/"
ALL_BUT_8_PATH = "Anon*.csv"

THE_8_PATH = "Week 8 Feedback (2016, incomplete) - results.csv"

### Weeks 1-7 (2016)

- NOTE: Data is _slightly_ different across various weeks and locations. **Approach with caution!**

In [4]:
all_but_8_2016_files = glob.glob(REL_PATH_DIRECTORY + ALL_BUT_8_PATH)

In [5]:
all_but_8_2016_files

['../datasets/SA_Feedback_Surveys_FINAL/2016/Anon Week 7 Feedback - Taipei.csv',
 '../datasets/SA_Feedback_Surveys_FINAL/2016/Anon Week 6 Feedback - Tokyo.csv',
 '../datasets/SA_Feedback_Surveys_FINAL/2016/Anon Week 1 Feedback - Singapore.csv',
 '../datasets/SA_Feedback_Surveys_FINAL/2016/Anon Week 7 Feedback - LA.csv',
 '../datasets/SA_Feedback_Surveys_FINAL/2016/Anon Week 4 Feedback - SF.csv',
 '../datasets/SA_Feedback_Surveys_FINAL/2016/Anon Week 5 Feedback - SV.csv',
 '../datasets/SA_Feedback_Surveys_FINAL/2016/Anon Week 4 Feedback - SG.csv',
 '../datasets/SA_Feedback_Surveys_FINAL/2016/Anon Week 6 Feedback - NY.csv',
 '../datasets/SA_Feedback_Surveys_FINAL/2016/Anon Week 5 Feedback - HK.csv',
 '../datasets/SA_Feedback_Surveys_FINAL/2016/Anon Week 1 Feedback - SF.csv',
 '../datasets/SA_Feedback_Surveys_FINAL/2016/Anon Week 2 Feedback - LA.csv',
 '../datasets/SA_Feedback_Surveys_FINAL/2016/Anon Week 6 Feedback - Taipei.csv',
 '../datasets/SA_Feedback_Surveys_FINAL/2016/Anon Week 3 F

## GOAL: Place _Week_ and _Location_ data in each 2016 (not Week 8) DF

In [6]:
filename = "Anon Week 1 Feedback - LA.csv"
fileparts = filename.split(" ")

test_file_path = REL_PATH_DIRECTORY + filename
df_test = pd.read_csv(test_file_path)

In [7]:
week_num, location = int(fileparts[2]), fileparts[5].split(".")[0]

In [8]:
df_garbage = pd.DataFrame([1, 2, 5, 3, 4, 1], columns=["trash_nums"])
df_garbage["Week"], df_garbage["Location"] = week_num, location
df_garbage

,trash_nums,Week,Location
0,1,1,LA
1,2,1,LA
2,5,1,LA
3,3,1,LA
4,4,1,LA
5,1,1,LA


In [9]:
 
# #Could set location and week columns manually to each file by setting a new DF for each filepath and running this function 
# def addLocationAndWeek(filepath, week, location):
#     location_and_week_arr
#     df = pd.read_csv(filepath)
#     df["Week"] = week
#     df["Location"] = place
#     return dataframe






# week = 48 - 53 
# import re
# # re.find()

# list_of_weeks = list()
# for filepath in all_but_8_2016_files:
#     week = (filepath[48:55])
#     list_of_weeks.append(week)
#     print(week)
    
    

In [10]:
def read_all_files():
    data_arr = list()
    for filename in all_but_8_2016_files:
        # When reading each file, we grab the Week & Location data and add it to two new columns
        week, location = _get_week_and_location(filename)
        df = pd.read_csv(filename)
        df["Week"], df["Location"] = week, location
        data_arr.append(df)
    return data_arr

def _get_week_and_location(filename):
    fileparts = filename.split(" ")
    week, location = int(fileparts[2]), fileparts[5].split(".")[0]
    return week, location

data = read_all_files()

### NOTE: The bad dataframe with `Unnamed: 0` is the `Anon Week 1 Feedback - SV.csv` DF

The `Unnamed: 0` column/feature is measuring Timestamp data (Datetime)

### NOTE: The bad dataframe with no `Timestamp` or `Unnamed: 0` is the `Anon Week 5 Feedback - SF.csv` DF

In [11]:
youre_the_one = None

for df in data:
    if "Unnamed: 0" not in df.columns and "Timestamp" not in df.columns:
        youre_the_one = df
        
youre_the_one

,What track are you in?,How would you rate your overall satisfaction with the Summer Academy this week?,How well is the schedule paced?,Week,Location
0,Apps,3,3,5,SF
1,Apps,4,3,5,SF
2,Apps,4,3,5,SF
3,Apps,4,4,5,SF
4,Apps,3,3,5,SF
5,Apps,5,3,5,SF
6,Apps,3,2,5,SF
7,Apps,5,3,5,SF
8,Apps,1,3,5,SF
9,Apps,4,3,5,SF


In [12]:
# week_one_through_seven_df = pd.DataFrame(index=index, columns=columns)
# Find a way to grab `week` and `location` data from filenames
# Find all potential unique columns
# Use unique columns to create master DF and put all data into that one (merging, copies)


column_names = dict()

for df in data:
    for column in df.columns:
        if column in column_names:
            column_names[column] += 1
        else:
            column_names[column] = 1
            
column_names

# for df in data:
#     week_one_through_seven_df = pd.concat(df)
    
# week_one_through_seven_df
# # week_one_through_seven_df .concat()

    
# week_one_through_seven_df + df
    
# week_one_through_seven_df

# rating = 'How would you rate your overall satisfaction with the Summer Academy this week?'
# data[data[rating]==4]

# dataframe["Week"] = week
# dataframe["Location"] = place



{'Timestamp': 37,
 'How would you rate your overall satisfaction with the Summer Academy this week?': 39,
 'How well is the schedule paced?': 33,
 'Week': 39,
 'Location': 39,
 'How well are the tutorials paced?': 6,
 'What track are you in?': 24,
 'Unnamed: 0': 1}

## TODO: Aggregate Schedule and Tutorial data into _Pacing_ column (mutual exclusion)

## Checking Each 2016 (not Week 8) DF for Unique Values

In [13]:
def check_unique_values_by_column(col_name):
    unique_values = set()
    for df in data:
        if col_name in df.columns:
            df_vals = df[col_name].unique().tolist()
            unique_values.update(df_vals)
    return unique_values

In [14]:
col_name_list = list(column_names.keys())
col_name_list

['Timestamp',
 'How would you rate your overall satisfaction with the Summer Academy this week?',
 'How well is the schedule paced?',
 'Week',
 'Location',
 'How well are the tutorials paced?',
 'What track are you in?',
 'Unnamed: 0']

### Create Dictionary of Unique Values per Column Name over All Data

In [15]:
uniques_dict = dict()
# For each column name in our list above, we create a dictionary element where...
# KEY: Column Name
# VALUE: All Unique Values for that Column Name across All Data
for index in range(8):
    col_name = col_name_list[index]
    uniques_dict[col_name] = check_unique_values_by_column(col_name)

In [16]:
uniques_dict["Location"]

{'HK', 'LA', 'NY', 'SF', 'SG', 'SV', 'Singapore', 'Taipei', 'Tokyo'}

In [17]:
data[0]
data[0]['How would you rate your overall satisfaction with the Summer Academy this week?'].values

array([3, 4, 4, 4, 5, 4, 4, 5, 5, 5, 3])

In [18]:
'''Map errors and np.NaN's to 0, then change datatype of column to type int '''
  
satisfaction_map = {
    '#REF!': 0,
    np.NAN: 0
}

satisfaction_column = 'How would you rate your overall satisfaction with the Summer Academy this week?' 
for df in data:
    if 'How would you rate your overall satisfaction with the Summer Academy this week?' in df:
        if '#REF!' in df[satisfaction_column]:
            df['How would you rate your overall satisfaction with the Summer Academy this week?'] = df['How would you rate your overall satisfaction with the Summer Academy this week?'].replace(satisfaction_map)
            pd.to_numeric(df['How would you rate your overall satisfaction with the Summer Academy this week?'])
        else:
            pd.to_numeric(df['How would you rate your overall satisfaction with the Summer Academy this week?'])
            
            


for df in data:
    if 'How would you rate your overall satisfaction with the Summer Academy this week?' in df:
        print(df['How would you rate your overall satisfaction with the Summer Academy this week?'].unique(), "\n\n")



# for df in data:
#     if 'How would you rate your overall satisfaction with the Summer Academy this week?' in df:
#         df[ 'How would you rate your overall satisfaction with the Summer Academy this week?'] = df[ 'How would you rate your overall satisfaction with the Summer Academy this week?'].map(rating_map)
#         pd.to_numeric(df['How would you rate your overall satisfaction with the Summer Academy this week?'])
# #         df.satisfaction_column_name = df.satisfaction_column_name.astype(int)


# df[satisfaction_column_name].unique()
data[0]
# [ 'How would you rate your overall satisfaction with the Summer Academy this week?']
#WHY is there still a nan?

ValueError: Unable to parse string "#REF!" at position 33

In [ ]:
# 2016 NOT Week 8 data dictionary mappings 

''' Mapping Schedule pacing to convert np.NaN and errors to 0 so that we can only deal with values greater than 0 in 
our calculations and making sure values are integers. '''

'''Our collection of mapping dictionaries to deal with np.NaN, #REF!, and wrong datatypes'''

schedule_pacing_map = {
    '#REF!': 0,
    np.NaN: 0, 
    1: 1, 
    2: 2, 
    '2': 2, 
    3: 3, 
    '3': 3, 
    '4': 4, 
    4: 4, 
    5: 5, 
    '5': 5
}


for df in data:
    if "How well is the schedule paced?" in df:
        df["How well is the schedule paced?"] = df["How well is the schedule paced?"].map(schedule_pacing_map)
        df = df.drop(df[df["How well is the schedule paced?"] > 0].index)


In [ ]:
for df in data:
    if "How well is the schedule paced?" in df:
        print(df["How well is the schedule paced?"].unique(), "\n\n")

In [ ]:
unique_types = set()

for item in check_unique_values_by_column(col_name_list[0]):
    unique_types.update([type(item)])
    
unique_types

### Example of Merging Mutually Exclusive Columns

In [ ]:
df1 = pd.DataFrame([1, 0, 3, 4, np.nan, np.nan, 0, 6, np.nan, np.nan, np.nan], columns=["a"])
df2 = pd.DataFrame([np.nan, np.nan, np.nan, np.nan, 5, 7, np.nan, np.nan, 2, 9, 8], columns=["b"])

Step 1: Replace all **NaNs**

In [ ]:
df1["a"] = df1["a"].fillna(0)
df2["b"] = df2["b"].fillna(0)

In [ ]:
df_final = df1['a'] + df2['b']

In [ ]:
s1, s2 = "New York", "No Location"
s1 + s2

In [ ]:
df_final

In [ ]:
pacing_name1 = 'How well is the schedule paced?'
pacing_name2 = 'How well are the tutorials pacecd?'

In [47]:
pacing_df =  pd.DataFrame(columns= ['How well is the schedule paced?','How well are the tutorials paced?'])

array_of_columns = list()


for df in data:
    if 'How well is the schedule paced?' in df.columns:
        array_of_columns.append(df['How well is the schedule paced?'])
        nan_df = pd.DataFrame(np.nan, index=len(df['How well is the schedule paced?'].size), columns='How well are the tutorials paced?')
        array_of_columns.append(nan_df)
#         pacing_df['How well is the schedule paced?'] = pacing_df['How well is the schedule paced?'] + df['How well is the schedule paced?']
    elif 'How well are the tutorials paced?' in df.columns:
        array_of_columns.append(df['How well are the tutorials paced?'])
        nan_df = pd.DataFrame(np.nan, index=len(df['How well are the tutorials paced?'].size), columns='How well are the tutorials paced?')
        array_of_columns.append(nan_df)
        
        
#         pacing_df[ 'How well are the tutorials paced?'] = pacing_df['How well are the tutorials paced?'] + df['How well are the tutorials paced?']
        
pacing_df

TypeError: object of type 'int' has no len()

In [45]:
height = 10
width = 20
df_0 = pd.DataFrame(0, index=range(height), columns=range(width))
len(df_0.index)

10

### Week 8 (2016)

In [ ]:
df_week_8 = pd.read_csv(REL_PATH_DIRECTORY + THE_8_PATH)

In [ ]:
df_week_8["Week"] = 8
df_week_8.rename(columns={"location": "Location"}, inplace=True)
df_week_8

### 2017

In [ ]:
FILEPATH = "../datasets/SA_Feedback_Surveys_FINAL/2017/Student_Feedback_Surveys_Superview.csv"
df_2017 = pd.read_csv(FILEPATH)

# 9 or 10 are promoters
# 7-8 are passives
# 0-6 are detractors

# df_2017.head()

# Clean Week data to remove redundant "week"

week_mapper = {
    "Week 1": 1,
    "Week 2": 2,
    "Week 3": 3,
    "Week 4": 4,
    "Week 5": 5,
    "Week 6": 6,
    "Week 7": 7,
    "Week 8": 8
}

df_2017["Week"] = df_2017["Week"].map(week_mapper)

In [ ]:
df_2017.head()

In [ ]:
df_2017 = pd.read_csv(FILEPATH)
df_2017 = df_2017[df_2017["Rating (Num)"] != "#ERROR!"]
df_2017["Rating (Num)"] = df_2017["Rating (Num)"].astype(int)

df_promoters = df_2017.loc[(df_2017['Rating (Num)'] >= 9) & (df_2017['Week'] == 'Week 7')]
df_passives = df_2017.loc[(df_2017['Rating (Num)'] >= 7) & (df_2017['Rating (Num)'] <= 8) & (df_2017['Week'] == 'Week 7')]
df_detractors = df_2017.loc[(df_2017['Rating (Num)'] < 7) & (df_2017['Week'] == 'Week 7')]


# df_promoters
# df_passives
# df_detractors   

# len(df_promoters) #78
# len(df_passives) #34
# len(df_detractors) #8

# num_of_promoters = len(df_promoters)
# num_of_promoters


## Two choices for Pro-Pas-Det Data Divisions:

- **Divide** up your promoters, passives, and detractors into _three_ independent DataFrames
- **Convert** your logic for promoter, passive, and detractor identifiation into _arguments_ that you can pass to your global DataFrame at anytime

---

# Data Manipulation

In [ ]:
promoter_count = 0
passive_count = 0
detractor_count = 0

index = []
columns = []

promoter_df = pd.DataFrame(index=index, columns=columns)
passive_df = pd.DataFrame(index=index, columns=columns)
detractor_df = pd.DataFrame(index=index, columns=columns)

In [ ]:
arg_promoter = (df["Rating (Num)"] >= 9)
promoters = df[arg_promoter]


# week_one = (promoters["Week"] == "Week 1")

# week_one


# len(promoters)
# promoters

In [ ]:
df2017.loc[:, ['ID', 'Track', 'Week', 'Rating (Num)']]


all_students = {}

for index, row in df.iterrows():
    
    